In [1]:
from utils import simulate_fair_price, track_pnl
from order_book import OrderBook
from trade import Trade

import plotly.express as px
import polars as pl
import numpy as np

## Question 1 : simulating fair prices

In [2]:
simulate_fair_price()

timestamp,Fair Price
datetime[μs],f64
2025-01-01 00:00:00,1.0
2025-01-01 01:00:00,1.000165
2025-01-01 02:00:00,1.001411
2025-01-01 03:00:00,1.003476
2025-01-01 04:00:00,1.009123
…,…
2025-12-31 20:00:00,0.001
2025-12-31 21:00:00,0.001
2025-12-31 22:00:00,0.001


Question for teacher : what is the economic reality behind these models / what are some good papers explaining these choices
- for a currency, going to 0.01 for a few months and then 50 months later is not realistic / signifies huge macroeconomic events
- what are some papers indicating which parameters make sense in this context

## Question 2 : Order book

In [3]:
# Example Usage
order_book = OrderBook(n_levels=5)
order_book.update_order(price=100.5, size=10, side="bid")
order_book.update_order(price=101.0, size=15, side="bid")
order_book.update_order(price=102.0, size=5, side="ask")
order_book.update_order(price=102.5, size=20, side="ask")
order_book.update_order(price=103.0, size=8, side="ask")

print("Best Bid:", order_book.get_best_bid())  # (101.0, 15)
print("Best Ask:", order_book.get_best_ask())  # (102.0, 5)
print("Order Book:\n", order_book.get_order_book())

Best Bid: (101.0, 15.0, datetime.time(21, 48, 52, 681498))
Best Ask: (102.0, 5.0, datetime.time(21, 48, 52, 682470))
Order Book:
 shape: (3, 6)
┌─────────────────┬──────────┬───────┬───────┬──────────┬─────────────────┐
│ timestamp_bid   ┆ size_bid ┆ bid   ┆ ask   ┆ size_ask ┆ timestamp_ask   │
│ ---             ┆ ---      ┆ ---   ┆ ---   ┆ ---      ┆ ---             │
│ time            ┆ f64      ┆ f64   ┆ f64   ┆ f64      ┆ time            │
╞═════════════════╪══════════╪═══════╪═══════╪══════════╪═════════════════╡
│ 21:48:52.681498 ┆ 15.0     ┆ 101.0 ┆ 102.0 ┆ 5.0      ┆ 21:48:52.682470 │
│ 21:48:52.672707 ┆ 10.0     ┆ 100.5 ┆ 102.5 ┆ 20.0     ┆ 21:48:52.682849 │
│ null            ┆ null     ┆ null  ┆ 103.0 ┆ 8.0      ┆ 21:48:52.683252 │
└─────────────────┴──────────┴───────┴───────┴──────────┴─────────────────┘


## Question 4 : Trade logic

In [4]:
trade = Trade(26, "buy")

In [5]:
order_book.get_order_book()

timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask
time,f64,f64,f64,f64,time
21:48:52.681498,15.0,101.0,102.0,5.0,21:48:52.682470
21:48:52.672707,10.0,100.5,102.5,20.0,21:48:52.682849
null,null,null,103.0,8.0,21:48:52.683252


In [6]:
ob = trade.update_orderbook_with_trade(orderbook=order_book)

In [7]:
ob.get_order_book()

timestamp_bid,size_bid,bid,ask,size_ask,timestamp_ask
time,f64,f64,f64,f64,time
21:48:52.681498,15.0,101.0,103.0,7.0,21:48:52.706999
21:48:52.672707,10.0,100.5,null,null,null


In [8]:
pl.DataFrame(trade.history)

side,price,size,timestamp
str,f64,f64,datetime[μs]
"""buy""",103.0,1.0,2025-04-15 21:48:52.707322
"""buy""",102.5,1.0,2025-04-15 21:48:52.707324
"""buy""",102.0,1.0,2025-04-15 21:48:52.707325


In [9]:
track_pnl(simulate_fair_price(), pl.DataFrame(trade.history), 10_000_000)

timestamp,pnl,inventory
datetime[μs],f64,f64
2025-04-15 21:48:52.707322,0.0,1e7
2025-04-15 21:48:52.707322,74.508623,9.999999e6
2025-04-15 21:48:52.707324,148.517247,9.999998e6
2025-04-15 21:48:52.707325,222.02587,9.999997e6
2025-04-15 21:48:52.707325,222.02587,9.999997e6
